In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/geotiffs-for-inference/sentinel-2-l2a_visual_S2A_MSIL2A_20250415T113641_R037_T28PET_20250415T191313.tif
/kaggle/input/mine-detection-resnet101-epochs-50-no-weights/Mine_Detection_Resnet101_Epochs_50_No_Weights.pth


In [2]:
pip install torchgeo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.0/605.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 855.8/855.8 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 72.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torchgeo
from torchgeo.datasets import RasterDataset,VectorDataset,stack_samples,unbind_samples, utils,random_bbox_assignment
from rasterio.crs import CRS
from torchgeo.samplers import RandomBatchGeoSampler,RandomGeoSampler,Units,GridGeoSampler
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import torch
import kornia.augmentation as K
import timeit
from rasterio.transform import from_bounds, from_origin
from rasterio.io import MemoryFile
import numpy as np
from rasterio.merge import merge
import rasterio
import torch
from torchvision.models.segmentation import deeplabv3_resnet101

model_path=r'/kaggle/input/mine-detection-resnet101-epochs-50-no-weights/Mine_Detection_Resnet101_Epochs_50_No_Weights.pth'

model = deeplabv3_resnet101(weights=None, num_classes=2)
model
from collections.abc import Callable
backbone = model.get_submodule('backbone')

conv = torch.nn.modules.conv.Conv2d(
    in_channels=3,
    out_channels=64,
    kernel_size=(7, 7),
    stride=(2, 2),
    padding=(3, 3),
    bias=False,
)
backbone.register_module('conv1', conv)

pred = model(torch.randn(8, 3, 64, 64))
pred['out'].shape
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_path,map_location=device,weights_only=False))
def create_in_memory_geochip(predicted_chip, geotransform, crs):
    """
    Apply georeferencing to the predicted chip.
    
    Parameters:
        predicted_chip (numpy array): The predicted segmentation chip (e.g., binary mask).
        geotransform (tuple): A tuple containing the geotransformation information of the chip (x-coordinate of the top left corner, x and y pixel size, rotation, y-coordinate of the top left corner, and rotation).
        crs (str): Spatial Reference System (e.g., EPSG code) of the chip.
        color_coded (bool): If the chip is color coded, it has a shape [H, W, C], othersize has a shape [1, H, W]

    Return:
        A rasterio dataset that is georreferenced.
    """
    photometric = 'MINISBLACK'
        
    memfile = MemoryFile()
    dataset = memfile.open(
        driver='GTiff',
        height=predicted_chip.shape[1],
        width=predicted_chip.shape[2],
        count=predicted_chip.shape[0],  # Number of bands
        dtype=np.uint8,
        crs=crs,
        transform=geotransform,
        photometric=photometric,
    )
    
    dataset.write(predicted_chip)
    return dataset

def georreferenced_chip_generator(dataloader, model, crs, pixel_size):
    """
    Apply georeferencing to the predicted chip.
    
    Parameters:
        dataloader (torch.utils.data.Dataloader): Dataloader with the data to be predicted.
        model (an https://github.com/qubvel/segmentation_models.pytorch model): model used for inference.
        crs (str): Spatial Reference System (e.g., EPSG code) of the chip.
        pixel_size (float): Pixel dimensoion in map units.

    Returns:
        A list of georeferenced numpy arrays of the predicted outputs.
    """
    georref_chips_list = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    for i, sample in enumerate(dataloader):
        image,bbox = sample['image'].to(device).float(), sample['bounds'][0]
        

        image = image/255. # as I'm not using a GeoDatamodule, I need to divide de images by 255 manually
        model.eval()

        with torch.no_grad():
            output = model(image)
        prediction=output['out']
         
        prediction = torch.softmax(prediction, dim=1)
        prediction = torch.argmax(prediction, dim = 1)

        geotransform = from_origin(bbox.minx, bbox.maxy, pixel_size, pixel_size)
        
        georref_chips_list.append(create_in_memory_geochip(prediction.cpu(), geotransform, crs))
    return georref_chips_list
def merge_georeferenced_chips(chips_list, output_path):
    """
    Merge a list of georeferenced chips into a single GeoTIFF file.

    Parameters:
        chips_generator (generator): A generator of Rasterio datasets representing the georeferenced chips.
        output_path (str): The path where the merged GeoTIFF file will be saved.

    Returns:
        None
    """
    # Merge the chips using Rasterio's merge function
    merged, merged_transform = merge(chips_list)
    
    # Calculate the number of rows and columns for the merged output
    rows, cols = merged.shape[1], merged.shape[2]

    # Update the metadata of the merged dataset
    merged_metadata = chips_list[0].meta
    merged_metadata.update({
        'height': rows,
        'width': cols,
        'transform': merged_transform
    })

    # Write the merged array to a new GeoTIFF file
    with rasterio.open(output_path, 'w', **merged_metadata) as dst:
        dst.write(merged)
        
    for chip in chips_list:
        chip.close()


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 225MB/s] 


In [5]:
test_dataset = RasterDataset(
    paths=r"/kaggle/input/geotiffs-for-inference/sentinel-2-l2a_visual_S2A_MSIL2A_20250415T113641_R037_T28PET_20250415T191313.tif",
)
test_sampler = GridGeoSampler(test_dataset, 2048, 2048)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, collate_fn=stack_samples)

pixel_size = test_dataset.res
if isinstance(pixel_size, tuple):
    pixel_size = float(pixel_size[0])  # or use np.mean(pixel_size) if needed
else:
    pixel_size = float(pixel_size)

crs = test_dataset.crs.to_epsg()

start = timeit.default_timer() # Measuring the time
chips_generator = georreferenced_chip_generator(test_dataloader, model, crs, pixel_size)
print("The time taken to predict was: ", timeit.default_timer() - start)

start = timeit.default_timer() # Measuring the time
file_name = r"/kaggle/working/predictions_Muliashi.tif"
merge_georeferenced_chips(chips_generator, file_name)
print("The time taken to generate a georrefenced image and save it was: ", timeit.default_timer() - start)

The time taken to predict was:  60.31786426300005
The time taken to generate a georrefenced image and save it was:  0.750606222999977
